<a href="https://colab.research.google.com/github/leearzuaga/Defense-vs-Position/blob/master/DefvsPosTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing/Extracting Data

In [1]:
# Dependencies and Settings

import requests
from bs4 import BeautifulSoup as BS 
import pandas as pd 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# URLs that holds the Running Back data I will be extracting

def_vs_rb_url = 'https://www.pro-football-reference.com/years/2019/fantasy-points-against-RB.htm'

In [3]:
''' This function sends a GET request to Pro Football Reference (PFR) that returns
a table in the form of HTML with defense vs position Fantasy Football (FF) statistics for all 32 NFL teams.
Finally, the HTML table is converted into tabluar form (DataFrame) using the read_html Pandas function.'''

def def_vs_pos(pos_url):
  res = requests.get(pos_url)
  soup = BS(res.content, 'lxml')

  table = soup.find('table')
  table_id = table.get('id')
  table = str(table)

  df = pd.read_html(table)[0]
  if df.columns.nlevels > 1:
    df.columns = df.columns.droplevel(level=0)
    df.fillna(0, inplace = True)

  return df

In [4]:
# Using our function above I created a def vs pos DataFrame (df) for the Running Back position.

rb_df = def_vs_pos(def_vs_rb_url) 

In [5]:
# Here I call rb_df to preview our running back's table
# Ultimately, I only care about Team, Games, RB fantasy points, and RB fantasy points per game
# In the following code we will create a function that will process the data to achieve this end

rb_df.head()

,Tm,G,Att,Yds,TD,Tgt,Rec,Yds,TD,FantPt,DKPt,FDPt,FantPt,DKPt,FDPt
0,Carolina Panthers,16,378,2011,27,81,61,392,3,436.42,510.4,466.9,27.3,31.9,29.2
1,Jacksonville Jaguars,16,357,1898,18,104,83,770,4,396.80,502.8,438.3,24.8,31.4,27.4
2,Washington Redskins,16,419,1972,11,125,96,809,5,370.10,484.1,418.1,23.1,30.3,26.1
3,Detroit Lions,16,393,1597,13,113,81,864,8,364.10,465.1,404.6,22.8,29.1,25.3
4,Miami Dolphins,16,433,1942,12,84,71,628,5,355.20,449.0,390.5,22.2,28.1,24.4


## Data Processing Function

In [6]:
# Drop columns we dont need

''' The intent of this function is to return the final processed version 
of our positional df. Because there's a lot going in within the body of this function, 
I will leave comments line by line as necessary.'''

def clean_rb_df(rb_df):

  # Line 12 drops a few columns from our rb_df and line 14 makes
  # that change to our rb_df permenant

  columns_to_drop = ['DKPt', 'FDPt', 'DKPt', 'FDPt'] 

  rb_df.drop(columns_to_drop, axis = 1, inplace = True)

  # Line 18 replaces Pandas default index which begins at 0 with the 'Tm' or Team column

  rb_df.set_index('Tm', inplace = True)

  # Line 22 renames our columns and gives them more descriptive names

  rb_df.columns = ['G', 'RushingAtt', 'RushingYDs', 'RushingTDs', 'Tgt', 'Rec', 'ReceivingYDs', 'ReceivingTDs', 'RB_FantasyPoints', 'RB_FantasyPoints/GM']

  # In lines 28-30 I recalculate the 'RB_FantasyPoints' and RB_FantasyPoints/GM columns by referencing other columns in out dataset. 
  # I do this so fantasy points accurately reflect the format that my league uses which is Point Per Reception or PPR.
  # Once we recalculate our new columns, we round those values to the tenths place.

  rb_df['RB_FantasyPoints'] = rb_df['RushingYDs'] / 10 + rb_df['RushingTDs'] * 6 + rb_df['Rec'] + rb_df['ReceivingYDs'] / 10 + rb_df['ReceivingTDs'] * 6
  rb_df['RB_FantasyPoints/GM'] = rb_df['RB_FantasyPoints'] / rb_df['G']
  rb_df['RB_FantasyPoints/GM'] = rb_df['RB_FantasyPoints/GM'].apply(lambda x: round(x, 1))

  # In line 34 I sort the df by values first in the 'RB_FantasyPoints/GM' column then the 'RB_FantasyPoints' column.

  rb_df.sort_values(['RB_FantasyPoints/GM', 'RB_FantasyPoints'], ascending = False, inplace = True)

  # In lines 39 and 40, I create a new column 'RB_Rank' which will rank all 32 NFL team
  # by the amount of fantasy points per game they give up to a position (in this case RB).

  rb_df['RB_Rank'] = rb_df['RB_FantasyPoints/GM'].rank()
  rb_df['RB_Rank'] = rb_df['RB_Rank'].apply(lambda x: round(x))

  # Line 44 sorts our rb_df by the values in the 'RB_Rank' column

  rb_df.sort_values('RB_Rank', ascending = False, inplace = True)

  # Finally we select the columns we want included in our final df

  rb_df = rb_df[['G', 'RB_FantasyPoints', 'RB_FantasyPoints/GM', 'RB_Rank']]

  return rb_df

In [7]:
# Here I pass my rb_df into the clean_rb_df function created above

rb_final_df = clean_rb_df(rb_df)

## Final Version Of Our RB DataFrame 

In [9]:
# Previewing our final df 
# Everything looks good

rb_final_df

,G,RB_FantasyPoints,RB_FantasyPoints/GM,RB_Rank
Tm,,,,
Jacksonville Jaguars,16,481.8,30.1,32
Carolina Panthers,16,481.3,30.1,32
Washington Redskins,16,470.1,29.4,30
Kansas City Chiefs,16,458.7,28.7,29
Detroit Lions,16,453.1,28.3,28
Houston Texans,16,444.8,27.8,27
Miami Dolphins,16,430.0,26.9,26
Green Bay Packers,16,425.5,26.6,25
Cincinnati Bengals,16,418.1,26.1,24


## Final Table With All Offensive Skill Positions

In [11]:
final_table = pd.read_csv('DEF_VS_ALL_POS.csv')

In [12]:
# Previewing the final table

final_table

,Tm,G,QB_FantasyPoints,QB_FantasyPoints/GM,QB_Rank,G.1,RB_FantasyPoints,RB_FantasyPoints/GM,RB_Rank,G.2,WR_FantasyPoints,WR_FantasyPoints/GM,WR_Rank,G.3,TE_FantasyPoints,TE_FantasyPoints/GM,TE_Rank
0,Arizona Cardinals,16,357.7,22.4,32,16,414.8,25.9,22,16,596.4,37.3,26,16,306.8,19.2,32
1,New York Giants,16,337.0,21.1,31,16,399.6,25.0,18,16,627.2,39.2,30,16,203.1,12.7,21
2,Miami Dolphins,16,334.9,20.9,30,16,430.0,26.9,26,16,627.4,39.2,30,16,192.1,12.0,15
3,Houston Texans,16,334.8,20.9,30,16,444.8,27.8,27,16,565.6,35.4,21,16,197.8,12.4,18
4,Oakland Raiders,16,331.2,20.7,28,16,392.2,24.5,16,16,544.1,34.0,17,16,211.9,13.2,23
5,Detroit Lions,16,327.5,20.5,27,16,453.1,28.3,28,16,646.6,40.4,31,16,189.2,11.8,14
6,Cincinnati Bengals,16,319.9,20.0,26,16,418.1,26.1,24,16,499.8,31.2,13,16,181.8,11.4,12
7,Washington Redskins,16,319.9,20.0,26,16,470.1,29.4,30,16,556.6,34.8,20,16,226.8,14.2,29
8,Tampa Bay Buccaneers,16,313.2,19.6,24,16,275.7,17.2,2,16,683.6,42.7,32,16,220.4,13.8,26
9,Atlanta Falcons,16,310.1,19.4,23,16,361.3,22.6,11,16,546.8,34.2,19,16,194.2,12.1,16
